# Final Projcet - HW5

In [ ]:
!nvidia-smi

Tue Dec  9 19:20:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   34C    P0             52W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# Environment Preparation
!pip install torchsummaryX==1.1.0
!pip install ruamel.yaml
!pip install torchmetrics[image]
!pip install wandb --quiet
!pip install pytorch_lightning
!pip install torchinfo --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 54.8 MB/s eta 0:00:00


In [ ]:
%matplotlib inline

import torch

import os

import time
import math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import torchsummaryX
import torch.nn as nn
import torch.nn.functional as F
import math
import gc
import glob
import wandb
import yaml
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import get_linear_schedule_with_warmup

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: ", DEVICE)


Device:  cuda


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# --- make sure your hw5_student_starter_code folder is in the Python path ---
import sys
path = "/content/gdrive/MyDrive/11685 Deep Learning/11685 Diffusion Probabilistic Models/hw5_student_starter_code"
if path not in sys.path:
    sys.path.append(path)
os.chdir(path)
print("✅ Current working dir:", os.getcwd())
!ls -la  # optional, shows files


✅ Current working dir: /content/gdrive/.shortcut-targets-by-id/1ibAv8MwbkbY-ow6IWKyjXfP_G9kp7Os5/11685 Diffusion Probabilistic Models/hw5_student_starter_code
total 378
drwx------ 2 root root   4096 Dec  9 19:06 configs
drwx------ 2 root root   4096 Oct 30 02:36 experiments
drwx------ 2 root root   4096 Oct 30 01:40 .idea
drwx------ 2 root root   4096 Oct 31 21:12 inference_epoch_9
drwx------ 2 root root   4096 Nov  1 03:32 inference_out
-rw------- 1 root root  11854 Dec  9 18:29 inference.py
drwx------ 2 root root   4096 Oct 31 20:57 inference_tiny
-rw------- 1 root root      0 Oct  6  2024 __init__.py
-rw------- 1 root root  11357 Oct  6  2024 LICENSE
drwx------ 2 root root   4096 Oct 30 01:40 models
drwx------ 2 root root   4096 Oct 30 01:40 pipelines
drwx------ 2 root root   4096 Nov 16 19:39 pretrained
drwx------ 2 root root   4096 Oct 31 19:42 __pycache__
-rw------- 1 root root   1694 Oct  6  2024 README.md
-rw------- 1 root root     38 Oct  6  2024 requirements.txt
drwx------ 2 

In [ ]:
# Hao
wandb.login(key="61f50d02f7b62af31be59681b63e436bb825a87f", relogin=True)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: haow5 (haow5-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
torch.cuda.empty_cache()
gc.collect()

91

In [ ]:
!mkdir -p /content/data
!tar -xvzf /content/gdrive/MyDrive/11685\ Deep\ Learning/11685\ Diffusion\ Probabilistic\ Models/data/imagenet100_128x128.tar.gz -C /content/data

Streaming output truncated to the last 5000 lines.
imagenet100_128x128/train/n01667778/n01667778_16828.JPEG
imagenet100_128x128/train/n01667778/n01667778_2905.JPEG
imagenet100_128x128/train/n01667778/n01667778_11595.JPEG
imagenet100_128x128/train/n01667778/n01667778_15965.JPEG
imagenet100_128x128/train/n01667778/n01667778_15959.JPEG
imagenet100_128x128/train/n01667778/n01667778_3425.JPEG
imagenet100_128x128/train/n01667778/n01667778_3557.JPEG
imagenet100_128x128/train/n01667778/n01667778_4119.JPEG
imagenet100_128x128/train/n01667778/n01667778_1660.JPEG
imagenet100_128x128/train/n01667778/n01667778_2259.JPEG
imagenet100_128x128/train/n01667778/n01667778_19903.JPEG
imagenet100_128x128/train/n01667778/n01667778_15342.JPEG
imagenet100_128x128/train/n01667778/n01667778_2621.JPEG
imagenet100_128x128/train/n01667778/n01667778_425.JPEG
imagenet100_128x128/train/n01667778/n01667778_3044.JPEG
imagenet100_128x128/train/n01667778/n01667778_21777.JPEG
imagenet100_128x128/train/n01667778/n01667778_1

In [ ]:
import random

root = "/content/data/imagenet100_128x128"
train_dir = os.path.join(root, "train")
val_dir   = os.path.join(root, "validation")

# All class names (folder names)
train_classes = [d for d in os.listdir(train_dir)
                 if os.path.isdir(os.path.join(train_dir, d))]
val_classes   = [d for d in os.listdir(val_dir)
                 if os.path.isdir(os.path.join(val_dir, d))]

# Classes that exist in both splits
common_classes = sorted(set(train_classes) & set(val_classes))

# Randomly choose 10 of them
random.seed(0)
selected_classes = random.sample(common_classes, 10)

print("Selected classes:", selected_classes)


Selected classes: ['n01773549', 'n02051845', 'n01775062', 'n01496331', 'n01728572', 'n01833805', 'n01824575', 'n01774384', 'n01739381', 'n01820546']


In [ ]:
import shutil

root = "/content/data/imagenet100_128x128"
splits = ["train", "validation"]

# your list of 10 classes
selected_classes = set(selected_classes)

backup_root = os.path.join(root, "excluded_classes")
os.makedirs(backup_root, exist_ok=True)

for split in splits:
    split_dir = os.path.join(root, split)
    backup_split = os.path.join(backup_root, split)
    os.makedirs(backup_split, exist_ok=True)

    for cls in os.listdir(split_dir):
        cls_path = os.path.join(split_dir, cls)
        if not os.path.isdir(cls_path):
            continue  # skip non-folder files

        if cls not in selected_classes:
            # move unwanted class folder to backup
            dst_path = os.path.join(backup_split, cls)
            print(f"Moving {cls_path} -> {dst_path}")
            shutil.move(cls_path, dst_path)

print("Done. train/ and validation/ now contain only the selected classes.")


Moving /content/data/imagenet100_128x128/train/n01751748 -> /content/data/imagenet100_128x128/excluded_classes/train/n01751748
Moving /content/data/imagenet100_128x128/train/n01677366 -> /content/data/imagenet100_128x128/excluded_classes/train/n01677366
Moving /content/data/imagenet100_128x128/train/n01968897 -> /content/data/imagenet100_128x128/excluded_classes/train/n01968897
Moving /content/data/imagenet100_128x128/train/n01978455 -> /content/data/imagenet100_128x128/excluded_classes/train/n01978455
Moving /content/data/imagenet100_128x128/train/n01818515 -> /content/data/imagenet100_128x128/excluded_classes/train/n01818515
Moving /content/data/imagenet100_128x128/train/n01944390 -> /content/data/imagenet100_128x128/excluded_classes/train/n01944390
Moving /content/data/imagenet100_128x128/train/n01985128 -> /content/data/imagenet100_128x128/excluded_classes/train/n01985128
Moving /content/data/imagenet100_128x128/train/n01955084 -> /content/data/imagenet100_128x128/excluded_classes/

In [ ]:
!python train.py \
  --config configs/ddim_10classes_cosine_ema_256batch.yaml

Streaming output truncated to the last 5000 lines.
 44% 44/100 [00:01<00:01, 36.97it/s]
 48% 48/100 [00:01<00:01, 36.91it/s]
 52% 52/100 [00:01<00:01, 36.97it/s]
 56% 56/100 [00:01<00:01, 36.89it/s]
 60% 60/100 [00:01<00:01, 36.78it/s]
 64% 64/100 [00:01<00:00, 36.58it/s]
 68% 68/100 [00:01<00:00, 36.33it/s]
 72% 72/100 [00:01<00:00, 36.15it/s]
 76% 76/100 [00:02<00:00, 35.99it/s]
 80% 80/100 [00:02<00:00, 36.09it/s]
 84% 84/100 [00:02<00:00, 35.39it/s]
 88% 88/100 [00:02<00:00, 35.58it/s]
 92% 92/100 [00:02<00:00, 35.71it/s]
 96% 96/100 [00:02<00:00, 35.54it/s]
100% 100/100 [00:02<00:00, 36.20it/s]
Checkpoint saved at experiments/exp-17-ddim_10classes_cosine_256batch_ema/checkpoints/checkpoint_epoch_last.pth
12/09/2025 23:30:03 - INFO - __main__ - Epoch 335/500
 67% 17035/25500 [3:59:45<9:57:27,  4.23s/it]12/09/2025 23:30:04 - INFO - __main__ - Epoch 335/500, Step 0/51, Loss 0.229611337184906 (0.229611337184906)
 67% 17085/25500 [4:00:16<1:23:06,  1.69it/s]
  0% 0/100 [00:00<?, ?it/s]

In [ ]:
!python inference.py \
  --config experiments/exp-16-ddim_10classes_cosine_512batch_ema/config.yaml \
  --ckpt experiments/exp-16-ddim_10classes_cosine_512batch_ema/checkpoints/checkpoint_epoch_last.pth > experiments/exp-16-ddim_10classes_cosine_512batch_ema/fid.txt 2>&1